In [2]:
def countWordsInBetween(str):
    a = len(str.split())
    return a

def endOfSentence (str): #checking if tag ends the sentence 
    if  str == "." or str == "!" or str == "?":
        return 1
    else:
        return 0

def endsWith (str): # checking if tag is followed by 's
    if str == "'s":
        return 1
    else: 
        return 0
    
def isAllUpperCase (str):
    a = 1
    words = str.split()
    for i in range(len(words)):
        if words[i][0] == "$" or words[i] == "von" or words[i] == "van" or words[i] == "da" or words[i] == "di" or words[i] == "de" or words[i] == "der" or words[i] == "ter" or words[i] == "al-" or words[i][0:3] == "al-" or words[i] == "ibn" :
            continue
        if words[i][0].isupper() != 1:
            a = 0
            break       
    return a

def checkArticles(str, index):
    if index == 1 and (str[index-1:index]=="a" or str[index-1:index] == "A"):
        return 1
    if index == 2 and (str[index-2:index] == "an" or str[index-2:index]) == "An":
        return 1
    if index > 2 and (str[index-3:index] == "the" or str[index-3:index] == "The" or str[index-2:index] == "an" or str[index-2:index] == "An" or str[index-1:index]=="a" or str[index-1:index]== "A"):
        return 1
    else:
        return 0        
def checkQuotations (str,startIndex):
    if startIndex > 0 and str[startIndex-1] == "\"":
        return 1
    else:
        return 0
    

In [9]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file = open ("reviews (115-159 only, with names tagged by Steven).csv","r")
newFile = open ("temp.csv","w")
data = file.read()
pos = data
count = data.count("<name>")
for i in range(count):
    startIndex = pos.index("<name>")
    endIndex = pos.index("</name>")
    s = checkArticles(pos, startIndex-1)
    newFile.write(str(s)+",")
    s = checkQuotations(pos,startIndex)
    newFile.write(str(s) +",")
    words = pos[startIndex+6:endIndex]
    numWords = countWordsInBetween(words)
    if numWords == 6:
        print(words+"name")
    newFile.write(str(numWords)+",")
    s = isAllUpperCase(words)
    newFile.write(str(s)+",")
    s = endsWith(pos[endIndex+7:endIndex+9])
    newFile.write(str(s)+",")
    s = endOfSentence(pos[endIndex+7:endIndex+8])
    newFile.write(str(s)+",")
    newFile.write(str(1)+"\n")
    pos = pos[endIndex+7:]
    
neg = data
count = data.count("<neg>")
for i in range(count):
    startIndex = neg.index("<neg>")
    endIndex = neg.index("</neg>")
    s = checkArticles(neg, startIndex-1)
    newFile.write(str(s)+",")
    s = checkQuotations(neg,startIndex)
    newFile.write(str(s) +",")
    words = neg[startIndex+5:endIndex]
    numWords = countWordsInBetween(words)
    if numWords == 7:
        print(words+"neg")
    newFile.write(str(numWords)+",")
    s = isAllUpperCase(words)
    newFile.write(str(s)+",")
    s = endsWith(neg[endIndex+6:endIndex+7])
    newFile.write(str(s)+",")
    s = endOfSentence(neg[endIndex+6:endIndex+7])
    newFile.write(str(s)+",")
    newFile.write(str(0)+"\n")
    neg = neg[endIndex+6:]
file.close()
newFile.close()


I Love You But I've Chosen Darknessneg
At the Chime of a City Clockneg


In [36]:
import numpy as np
import csv
import pandas as pd
from sklearn import tree
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
filename = 'trainSet.csv'
names = ["article","quotation","numWords","capital","s","endOfSent","label"]
data = pd.read_csv(filename, names=names)
Xtrain = np.array(data[names[:-1]][1:]).astype(int)
Ytrain = np.array(data["label"][1:]).astype(int)
parameters = {'max_depth':range(3,10)}
clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, cv = 100, n_jobs=4)
clf.fit(Xtrain, Ytrain)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_)
#print(data[names[:-1]])

0.909490119541 {'max_depth': 5}


In [132]:
from sklearn.feature_selection import VarianceThreshold
#X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(Xtrain)
print(Xtrain)

[[0 0 2 1 0 1]
 [0 0 1 1 0 1]
 [0 0 1 1 1 0]
 ..., 
 [0 0 1 0 0 0]
 [0 0 2 0 0 0]
 [0 0 2 1 0 0]]


In [38]:
from sklearn.metrics import precision_recall_fscore_support as score

precision, recall, fscore, support = score(Ytrain, predicted)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: [ 0.96819604  0.85786337]
recall: [ 0.85694509  0.9684265 ]
fscore: [ 0.90917993  0.9097982 ]
support: [2167 1932]


In [120]:
from sklearn import linear_model
regr = linear_model.LogisticRegression()
#regr.fit(Xtrain, ytrain)
#predictions = regr.predict(X_test)
parameters = {'C':[0.1]}
clf = GridSearchCV(regr, parameters,cv = 100, n_jobs=1)
clf.fit(Xtrain, Ytrain)
logreg_model = clf.best_estimator_
print (clf.best_score_,clf.best_params_)

0.90680653818 {'C': 0.1}


In [121]:
from sklearn.metrics import precision_recall_fscore_support as score
predicted = cross_val_predict(logreg_model, Xtrain, Ytrain,cv = 10)
precision, recall, fscore, support = score(Ytrain, predicted)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: [ 0.98479913  0.8435977 ]
recall: [ 0.83710198  0.98550725]
fscore: [ 0.90496383  0.90904751]
support: [2167 1932]


In [130]:
from sklearn.ensemble import RandomForestClassifier
#regr.fit(Xtrain, ytrain)
#predictions = regr.predict(X_test)
parameters = {'n_estimators':range(2,25)}
clf = GridSearchCV(RandomForestClassifier(), parameters,cv = 100, n_jobs=1)
clf.fit(Xtrain, Ytrain)
rfreg_model = clf.best_estimator_
print (clf.best_score_,clf.best_params_)

0.908270309832 {'n_estimators': 4}


In [131]:
from sklearn.metrics import precision_recall_fscore_support as score
predicted = cross_val_predict(rfreg_model, Xtrain, Ytrain,cv = 100)
precision, recall, fscore, support = score(Ytrain, predicted)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: [ 0.96070321  0.85727483]
recall: [ 0.85740655  0.96066253]
fscore: [ 0.90612046  0.9060288 ]
support: [2167 1932]


In [55]:
from sklearn import svm
from sklearn.model_selection import StratifiedShuffleSplit
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
#parameters = {'C': np.logspace(-2, 10, 13),'gamma': np.logspace(-9, 3, 13)}
#cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
#clf = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=10)
clf = GridSearchCV(svm.SVC(), parameters, cv=10)
clf.fit(Xtrain, Ytrain)
svm_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_)

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import GridSearchCV

features = ["article","quotation","numWords","capital","s","endOfSent","label"]

df = pd.read_csv('trainSet.csv',header=None,names=features)
#df['class'] = df['class'].map({'g':0,'h':1})

x = df[features[:-1]]
x
#y = df['label']

#parameters = {'max_depth':range(1,5)}
#clf = tree.DecisionTreeClassifier()
#clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4)
#clf.fit(x, y)
#tree_model = clf.best_estimator_
#print (clf.best_score_, clf.best_params_) 

In [53]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_fscore_support as score
svmpredicted = cross_val_predict(svm_model, Xtrain, Ytrain,cv = 10)

precision, recall, fscore, support = score(Ytrain, svmpredicted)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: [ 0.97353097  0.85158371]
recall: [ 0.84863867  0.97412008]
fscore: [ 0.90680473  0.90873974]
support: [2167 1932]


In [122]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xtrain,Ytrain, test_size=0.4, random_state=0)
print(X_test.shape)
print(X_train.shape)

(1640, 6)
(2459, 6)
